---
title: "Hyperspectral unmixing"
author: "Louie John M. Rubio"

bibliography: hyperspec_notes.bib
csl: ieee.csl

numbersections: true

format:
    html:
        code-fold: true
        embed-resoures: true
        self-contained: true

jupyter: python3
toc: true
---

Notes for Physics 312 class readings

## General overview/ideas

Hyperspectral cameras capture hyperspectral signals which can have mixed pixels due to the limited spatial resolution of the instrument. The signal measured at a pixel is a mixture of light scattered by materials in the scene within the pixel's equivalent spatial resolution. @fig-model shows an example of spectra for different pixels, both mixed and pure.

The main goal for hyperspectral unmixing is to separate pixel spectra to the spectral signatures of the materials in the pixel (endmembers) and their relative amounts (abundances).

![Hyperspectral imaging example. From Dalla Mura's slides [@dalla_mura]](dalla_mura.png){width=80% #fig-model}


### Mixing models
Unmixing algorithms depend on the type of mixing model assumed for the hyperspectral images. They are usually classified as either **linear** or **nonlinear**.
    
* linear - Macroscopic pure components are distributed in patches within the field of view (or the pixel). 
    
    - The measured spectra at a pixel is the weighted average of the individual spectra of the materials in the pixel.

* nonlinear - Microscopic pure components are intimately mixed within the pixel. Another possibility is when the scene has multiple layers, resulting in complex mechanisms for scattering light.
    
    - This usually requires a priori knowlege on the materials in the scene.
    
    
Without a priori knowledge, the problem at hand can be referred to as blind/unsupervised hyperspectral unmixing (blind HU).

* The goal is still to identify materials present in a scene (and their compositions) using high spectral resolution of hyperspectral images.

* Convex geometry concepts (discovered empirically) find a lot of usage. 

    
In the paper "A Signal Processing Perspective on Hyperspectral Unmixing" [@remote_sense], the focus is on linear mixing models. The notes that follow here are mainly from this paper and Dalla Mura's slides [@dalla_mura].

## Linear mixing model

The linear mixing model is relatively simplistic but is a very representative model, which is the focus of many blind HU developments.

While the linear mixing model can fail for strongly non-linear systems, it is still considered to be acceptable for most real-world scenarios. 

With $y_m[n]$ being the hyperspectral measurement for a spectral band $m$ and pixel $n$: let $y[n] = [y_1[n], y_2[n],...,y_M[n]]^T \in \mathbb{R}^M$ where M is the number of spectral bands. The linear mixing model can be expressed as:

$$ y[n] = \sum^N_{i = 1}a_i s_i[n] + v[n] = \mathbf{A} s[n] + v[n] $$

for $n = 1,...,L$ (meaning there are L pixels. $a_i \in \mathbb{R}^M$ is an endmember signature vector, which contains spectral components of a specific material in the scene. N is the number of endmembers (or materials) in the scene. $\mathbf{A} = [a_1,...a_N] \in \mathbb{R}^{M\times N}$ is the endmember matrix. s_i[n] is the contribution of each material $i$ at pixel $n$, or the abundance vector. and $v[n]$ is the noise.

Other aspects regarding linear mixing model formulation:

- M can be larged because of the fine spectral resolution of hyperspectral cameras

- Mixing is a consequence of spatial resolution limits

- $s_i[n]$ is normalized (**abundance sum constraint or sum-to-one constraint**)

- Abundances are non-negative.

- Endmember spectra cannot be negative (since reflectance cannot be negative)

Complications:

- $y[n]$ are processed measurements. (raw measurements from the camera already go through processing steps)

- For simplicity, endmembers are associated with materials (assumed purity)
    
    - Definition of endmembers can be subjective depending on the application.
    
- Sum-to-one constraint is assumed to hold.

### Problem formulation

Recovering the abundance map of every material is the goal.
    
- If the endmember matrix $A$ is known (materials in the scene are known exactly), unmixing becomes a constrained least squares problem

- Usually, $A$ is not fully known because in reality, all the materials in the scene are not known exactly. 

- The problem is formulated similarly to blind source separation in formulation.

    - BSS methods are not in any mainstream HU approaches because under the unit simplex constraint, for the feasible set of abundance vectors do not satisfy statistical independence assumption.

- Identifying the number of endmembers is considered a separate problem. ($N$ is generally assumed)

## General scheme for hyperspectral unmixing

From Dalla Mura slides [@dalla_mura]

1) Estimation of the number of endmembers $N$
2) Endmember extraction 
3) Estimation of abundances for all pixels


## Methods for subspace estimation
* Principal component analysis (PCA) is used to estimate the number of significant components to be used.

* Minimum noise fraction orders components using signal-to-noise ratio.

* Hyperspectral subspace identification minimum error (HySime)
 
    - finds $k$ eigenvectors that contains most of the information
 
    - done by minimizing the MSE between the original data and a projection into the eigenvector subspace


## Endmember extraction

In categorizing algorithms for endmember extraction, there are three situations that can be encountered [@dalla_mura]:

1) The data contains at least one pure pixel per endmember. (Pure pixel assumption)
2) The data does not contain pure pixels but contains N - 1 spectral vectors on each facet. (Minimum volume simplex fit)
3) For highly mixed data, there are no spectral vectors near the facets. Volume minimization fails here. (Statistical framework must be used)

### Pure pixel assumption

Under the pure pixel assumption, an endmember $i$ has a pure pixel for some index $l_i$ if $$s[l_i] = \hat{e}_i$$ 

$\hat{e}_i$ is a unit vector with a nonzero element only for the $i$th entry. Pure pixel assumption holds if every endmember has a pure pixel. [@remote_sense]

In @fig-model, the pixel for water is a pure pixel because there are no other materials within.

#### Methods that use the pure pixel assumption

1) Successive projections algorithm (SPA)
    - Finding endmember signatures by successive nulling.
    
    - SPA finds all endmembers for the noiseless case and under PPA.
    
    - If endmembers are highly similar, SPA will have problems.
    
    - likely similar to orthogonal subspace projection (OSP) in Dalla Mura's slides

2) Vertex component analysis
    - Similar to SPA (in employing successive nulling), but picks pure pixels differently.
    
    - VCA iteratively projects data to a direction orthogonal to the subspace spanned by endmembers that have been found already
    
    - Main difference with OSP is a noise characterization to reduce sensitivity to noise
    
    - Done using singular value decomposition (SVD).

Notes for VCA and SPA:

- Dimensionality reduction (DR) is usually done as preprocessing before doing pure pixel search

- DR reduces noise

3) N-FINDR algorithm 

    - Assumes pure pixels in the scene. Maximizes the volume formed by the pixel vectors.
 
4) SVMAX (successive volume maximization)

    - Done by maximizing the simplex volume formula.
    
### Convex geometry    
For the last two methods, we've noticed that it talks about maximizing a simplex volume. What does this mean in terms of hyperspectral signals? For the previous 2 algorithms and the other algorithms will be discussing, we will use the convex geometry of hyperspectral signals

![Convex geometry of hyperspectral signals. From "A Signal Processing Perspective on Hyperspectral Unmixing" [@remote_sense]](convex_geom.png){width=80% #fig-conv}

In @fig-conv, we see that for N = 3, the convex hull of the endmember signatures $a_i$ forms a triangle that encloses all $y[n]$ (or all the hyperspectra of the pixels). Convex geometry HU methods finds a set of vectors $\hat{a_i}$ that creates a convex hull that closely matches the convex hull created by the true endmembers $a_i$ (affine set fitting problem).

#### Methods without the pure pixel assumption
These methods generate virtual endmembers located at the simplex with minimum volume enclosing all observations if *pure signatures are not assumed to be present*. Thus, these algorithms are just different implementations of simplex volume minimization. 

##### Minimum volume algorithms
Simplex volume minimization approaches fits the simplex by finding a simplex that fits all the pixels. This is robust against lack of endmember pixels.

- Assuming pure pixel assumption holds, the optimal solution for simplex volume minimization ends up is the true endmembers’ signatures.

1) Minimum volume simplex analysis (MVSA)[@MVSA]
    
    - Considered a hard nonconvex optimization algorithm. 
    
    - Initialized with an inflated version of the simplex using VCA to avoid being trapped in local minima.
    
    - All data points will be in the simplex.
    
    - By changing the positivity constraints, it will be more robust to noise and outliers
    
2) Simplex identification via split augmented Lagrangian (SISAL)

    - Positivity constraints are replaced by hinge-type soft constraints which makes the it robust to outliers and noise
    
    - Has algorithmic complexity of $O(np)$ ($n$ spectral vectors, $p$ endmembers), making it much faster than other algorithms considered state-of-the-art


### Statistical methods
For highly mixed data (where there are no pixels near the endmember signatures), minimizing the volume fails. The unmixing problem is posed a statistical inference problem.

## Abundance estimation
The last step in the unmixing problem is determining the abundance for the pixels. Mostly from Dalla Mura's slides [@dalla_mura]

1) Unconstrained least squares
    - Used when the number of endmembers and the endmember signatures are known

2) Non-negative constrained least squares
    - Used when the abundance non-negativity constraint needs to be satisfied
    
    - Increases the computational complexity of the abundance estimation
    
    - Solved using quadratic programming
    
3) Fuly constrained least squares

    - Both the sum-to-one constraint and non-negativity constraint are imposed
    
Imposing the non-negativity constraint is usually more important than the sum-to-one constraint. When endmembers are unknown, they usually need to be computed first.

Algorithms that do not follow the pure pixel assumption simultaneously calculates the abundance estimates.

## Other methods

1) Dictionary based semi-blind HU [@remote_sense]

-  Sparse regression 
    
    - uses a library of spectral signatures
    
    - does not require knowledge of N (number of N members)
    
    - main disadventage is being a combinatorial problem, since the endmember signatures come from a library (that's why it's dictionary based) [@dalla_mura]

2) Nonnegative matrix factorization (NMF)[@remote_sense]
    
- Originally a dimensionality reduction tool for environmental data/data mining
    
- Low-rank matrix approximation problem

- The NMF factors obtained can serve as estimates for the endmembers and abundances.

## Pareto task inference (ParTi) method

So far, the motivation for hyperspectral unmixing results from dealing with high-dimensional data from hyperspectral cameras. Essentially, hyperspectral unmixing is a method that can deal with high-dimensional data.

The Pareto task inference (ParTi) method is used mainly on large biological datasets. This is based on Pareto optimality of organisms [@Parti]. 

Pareto optimality predicts that organisms that need to perform multiple tasks have phenotypes that fall in a simplex [@Parti], much like how pixels of hyperspectral images fall within a simplex formed by endmember signatures. Phenotypes optimal for a task are called archetypes, which are vertices of the simplex.

By this definition, all phenotypes within the simplex can be described as a combination of the archetypes. Finding endmember signatures and archetypes are similar problems, mathematically.

Uri Alon's group packages 5 algorithms for finding the archetypes from the data, which includes SISAL [@SISAL] and MVSA[@MVSA] (methods employed for signal processing!).

## Applications on other datasets

I first heard of hyperspectral unmixing in de Castro et al. [@SPP_cricri]'s paper applying hyperspectral unmixing on precinct voting data to get voting archetypes from the dataset. The pixel spectra are the clustered precinct-level data for 2013 to 2019, and the bands of the spectra are the fraction of votes over the total number of ballots of each candidate.

The archetypes are here are the voting patterns (which combination of voters are voted), and the archetypes are interpreted based on the patterns that appear.

### Other questions in class discussion
Questions:
- Do endmembers and archetypes refer to the same thing?
    - No (endmember signatures and archetypes are the ones that are similar)
- Is OSP the same as the successive projections algorithm(in the first PDF)
    - Looks like it
- Simplex volume minimization techniques seem to be sensitive to noise (slide 41 of Dalla Mura's slides [@dalla_mura]). How much noise is enough to affect the algorithms here?
    - Depends on the application. Assumptions are made when removing the data.

## Key takeaways
- Hyperspectral unmixing aims to to get the constituent spectra (endmember signatures) of endmembers and their relative amounts (abundance) for a combined spectra (the pixel spectra).

- For most real-world applications, the linear mixing model should work as long as the non-linearities in the system are negligible. The linear mixing model assumes that all abundances sum-to-one and are non-negative. The spectra should also be non-negative.

- Outside of its use in processing hyperspectral signals, hyperspectral unmixing has found use in other domains where there is a need to evaluate high-dimensional datasets.

## References

::: {#refs}
:::